# Protein-Protein Osmotic Second Virial Coefficient Calculation

This takes two input PDB structures and calculates the potential of mean force between them, averaging over the angular space using a regular grid. The
Calvados 3 parameters for the pair-wise interaction between coarse grained amino acids is used.
The calculation is done using the still experimental `virialize` tool, written
in Rust. It is part of the yet unreleased Rust version of Faunus. Use with care!

_M. Lund, December 2024_

In [1]:
# @title
!pip install virialize mdtraj --quiet
import mdtraj as md
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np
import os
import ipywidgets as widgets
from IPython.display import display
%matplotlib inline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.4 MB/s eta 0:00:00


In [2]:
# @title
def convert_pdb(pdb_file, output_xyz_file):
    ''' Convert PDB to coarse grained XYZ file; one bead per amino acid '''
    traj = md.load_pdb(pdb_file, frame=0)
    residues = []
    for res in traj.topology.residues:
        if not res.is_protein:
            continue
        cm = [0.0, 0.0, 0.0]  # residue mass center
        mw = 0.0 # residue weight
        for a in res.atoms:
            cm = cm + a.element.mass * traj.xyz[0][a.index]
            mw = mw + a.element.mass
        cm = cm / mw * 10.0
        residues.append(dict(name=res.name, cm=cm))
    with open(output_xyz_file, "w") as f:
        f.write(f'{len(residues)}\n')
        for i in residues:
            f.write(f"{i['name']} {i['cm'][0]} {i['cm'][1]} {i['cm'][2]}\n")
    print(f"Converted {pdb_file} -> {output_xyz_file} with {len(residues)} residues.")

## Generate Force Field

The following creates the two files `calvados3.yaml` and `topology.taml` which sets up the force field. By default this provides a coarse grained representation of amino acids: one bead or interaction site per residue. You may customize to other representations if you wish.

In [3]:
# @title
%%writefile calvados3.yaml
comment: "Calvados 3 coarse grained amino acid model. Charges fixed to generic, neutral pH"
version: 0.1.0
atoms:
  - {charge: 1.0,  hydrophobicity: !Lambda 0.7407902764839954, mass: 156.19, name: ARG, σ: 6.56, ε: 0.8368}
  - {charge: -1.0, hydrophobicity: !Lambda 0.092587557536158,  mass: 115.09, name: ASP, σ: 5.58, ε: 0.8368}
  - {charge: 0.0,  hydrophobicity: !Lambda 0.3706962163690402, mass: 114.1,  name: ASN, σ: 5.68, ε: 0.8368}
  - {charge: -1.0, hydrophobicity: !Lambda 0.000249590539426,  mass: 129.11, name: GLU, σ: 5.92, ε: 0.8368}
  - {charge: 1.0,  hydrophobicity: !Lambda 0.1380602542039267, mass: 128.17, name: LYS, σ: 6.36, ε: 0.8368}
  - {charge: 0.0,  hydrophobicity: !Lambda 0.4087176216525476, mass: 137.14, name: HIS, σ: 6.08, ε: 0.8368}
  - {charge: 0.0,  hydrophobicity: !Lambda 0.3143449791669133, mass: 128.13, name: GLN, σ: 6.02, ε: 0.8368}
  - {charge: 0.0,  hydrophobicity: !Lambda 0.4473142572693176, mass: 87.08,  name: SER, σ: 5.18, ε: 0.8368}
  - {charge: 0.0,  hydrophobicity: !Lambda 0.5922529084601322, mass: 103.14, name: CYS, σ: 5.48, ε: 0.8368}
  - {charge: 0.0,  hydrophobicity: !Lambda 0.7538308115197386, mass: 57.05,  name: GLY, σ: 4.5,  ε: 0.8368}
  - {charge: 0.0,  hydrophobicity: !Lambda 0.2672387936544146, mass: 101.11, name: THR, σ: 5.62, ε: 0.8368}
  - {charge: 0.0,  hydrophobicity: !Lambda 0.3377244362031627, mass: 71.07,  name: ALA, σ: 5.04, ε: 0.8368}
  - {charge: 0.0,  hydrophobicity: !Lambda 0.5170874160398543, mass: 131.2,  name: MET, σ: 6.18, ε: 0.8368}
  - {charge: 0.0,  hydrophobicity: !Lambda 0.950628687301107,  mass: 163.18, name: TYR, σ: 6.46, ε: 0.8368}
  - {charge: 0.0,  hydrophobicity: !Lambda 0.2936174211771383, mass: 99.13,  name: VAL, σ: 5.86, ε: 0.8368}
  - {charge: 0.0,  hydrophobicity: !Lambda 1.033450123574512,  mass: 186.22, name: TRP, σ: 6.78, ε: 0.8368}
  - {charge: 0.0,  hydrophobicity: !Lambda 0.5548615312993875, mass: 113.16, name: LEU, σ: 6.18, ε: 0.8368}
  - {charge: 0.0,  hydrophobicity: !Lambda 0.5130398874425708, mass: 113.16, name: ILE, σ: 6.18, ε: 0.8368}
  - {charge: 0.0,  hydrophobicity: !Lambda 0.3469777523519372, mass: 97.12,  name: PRO, σ: 5.56, ε: 0.8368}
  - {charge: 0.0,  hydrophobicity: !Lambda 0.8906449355499866, mass: 147.18, name: PHE, σ: 6.36, ε: 0.8368}

Writing calvados3.yaml


In [4]:
# @title
%%writefile topology.yaml
include: [calvados3.yaml]
system:
  energy:
    nonbonded:
      # Note that a Coulomb term is automatically added, so don't specify one here!
      default:
        - !AshbaughHatch {mixing: arithmetic, cutoff: 20.0}

Writing topology.yaml


# Upload PDB files

Here you must upload two PDB files with atomistic information about the globular proteins to be investigated. These will be coarse grained to an amino acid representation, matching the Calvados 3 model. Currently this is not very sophisticated -- see included Python function for details.

_If you run into a stack-related error for larger PDB files, try switching to another browser. This is a known problem with CoLab._

In [ ]:
# @title
xyzfile1 = files.upload()
xyzfile2 = files.upload()
if len(xyzfile1.keys()) != 1 or len(xyzfile2.keys()) != 1:
    raise ValueError("Please upload two single PDB files.")

pdb_files = [xyzfile1.keys(), xyzfile2.keys()]
for ndx, pdbfile in enumerate(pdb_files):
  for fn in pdbfile:
    if not fn.endswith(".pdb"):
      raise ValueError("PDB files required")
    convert_pdb(fn, f"mol{ndx+1}.xyz")

In [ ]:
# @title Input parameters { display-mode: "form" }

# @markdown #### Ionic strength (mol/l):
ionic_strength = 0.1 # @param {type:"number"}
# @markdown #### Temperature (kelvin):
temperature = 298.15 # @param {"type":"number"}
# @markdown #### Angular resolution (radians):
angular_resolution = 1 # @param {"type":"slider","min":0.1,"max":1.2,"step":0.1}
# @markdown #### Minimum mass-center separation (angstrom):
min_distance = 10 # @param {"type":"number"}
# @markdown #### Maximum mass-center separation (angstrom):
max_distance = 60 # @param {"type":"number"}
# @markdown #### Mass-center separation steps (angstrom):
distance_step = 2 # @param {"type":"number"}

!virialize scan --icotable \
--mol1 mol1.xyz --mol2 mol2.xyz \
--top topology.yaml \
--molarity {ionic_strength} \
--resolution {angular_resolution} \
--rmin {min_distance} --rmax {max_distance} --dr {distance_step} \
--temperature {temperature} \
--pmf pmf.dat

r, w, u = np.loadtxt("pmf.dat", unpack=True)
plt.plot(r, w, label="Free energy", color="orange", lw=4, alpha=0.7)
plt.plot(r, u, label="Mean energy", color="red", lw=4, alpha=0.7)
plt.legend(loc=0, frameon=False)
plt.xlabel('Mass center separation (angstrom)')
plt.ylabel('Energy (kT)')

# Experimental section using `ipywidget`

This is incomplete and cannot be used.

In [ ]:
# @title
resolution_slider = widgets.FloatSlider(
    value=1.0,
    min=0.1,
    max=1.2,
    step=0.1,
    description='Angular resolution:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)

interval_slider = widgets.FloatRangeSlider(
    value=[25, 50],
    min=0,
    max=120.0,
    step=1.0,
    description='Distances (Å)',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f',
)
pdb_upload1 = widgets.FileUpload(accept='.pdb', multiple=False)
pdb_upload2 = widgets.FileUpload(accept='.pdb', multiple=False)

display(widgets.HBox([widgets.Label(value="1st PDB:"), pdb_upload1]))
display(widgets.HBox([widgets.Label(value="2nd PDB:"), pdb_upload2]))
display(interval_slider)
display(resolution_slider)

button = widgets.Button(description="Run!")
output = widgets.Output()

def on_button_clicked(b):
  # Display the message within the output widget.
  with output:
    print("Button clicked.")

button.on_click(on_button_clicked)
display(button, output)